In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import warnings

warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


# defining the model

In [3]:
class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32).to(device)
        self.y = torch.tensor(y, dtype=torch.long).to(device)
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [4]:
class MAML(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MAML, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_samples = 6
        self.epochs = 20
        self.alpha = 0.05  
        self.beta = 0.01  # meta learning rate
        self.theta = nn.Parameter(torch.randn(input_dim, output_dim).to(device))
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        a = torch.matmul(x, self.theta)
        return self.softmax(a)

    def sample_points(self, k, x, y):
        indices = np.random.choice(len(x), k)
        return x[indices], y[indices]

    def train(self, x_train, y_train, x_val, y_val):
        train_dataset = CustomDataset(x_train, y_train)
        train_loader = DataLoader(train_dataset, batch_size=self.num_samples, shuffle=True)

        optimizer = optim.Adam(self.parameters(), lr=self.alpha)

        for e in range(1, self.epochs + 1):
            self.theta_ = []
            for x_batch, y_batch in train_loader:
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)

                y_hat = self.forward(x_batch)
                y_batch_encoded = torch.eye(self.output_dim, device=device)[y_batch]
                loss = -torch.mean(y_batch_encoded * torch.log(y_hat + 1e-7))

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                self.theta_.append(self.theta.detach().clone())

            meta_gradient = torch.zeros_like(self.theta, dtype=torch.float32).to(device)
            for i in range(self.num_samples):
                x_test, y_test = self.sample_points(10, x_train, y_train)
                x_test = torch.tensor(x_test, dtype=torch.float32).to(device)
                y_pred = self.forward(x_test)
                y_test_encoded = torch.eye(self.output_dim)[y_test].to(device)
                meta_gradient += torch.matmul(x_test.T, (y_pred - y_test_encoded)) / self.num_samples

            self.theta.data -= self.beta * meta_gradient

            with warnings.catch_warnings():
                warnings.filterwarnings("ignore", category=UserWarning)
                x_val = torch.tensor(x_val, dtype=torch.float32).to(device).clone().detach().requires_grad_(True)
            y_val_pred = self.forward(x_val)
            val_loss = -torch.mean(torch.eye(self.output_dim, device=device)[y_val] * torch.log(y_val_pred + 1e-7))

    def predict(self, x):
        with torch.no_grad():
            x = torch.tensor(x, dtype=torch.float32).to(device)
            y_pred = self.forward(x)
            _, predictions = torch.max(y_pred, dim=1)
            return predictions.cpu().numpy()


In [5]:
data = pd.read_csv('text_embeddings_train.csv')
x = data['text_embedding'].str.split('\t', expand=True).astype(float).values

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['label'])

In [6]:
len(data)

43793

In [7]:
num_labels = len(label_encoder.classes_)
print(num_labels)

6


In [8]:
models = []
accuracies = []

# training

In [11]:
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.8,stratify=y, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5,stratify=y_temp, random_state=42)

model = MAML(input_dim=x.shape[1], output_dim=num_labels).to(device)

model.train(x_train, y_train, x_val, y_val)

label_accuracy = {}
for label_idx, label in enumerate(label_encoder.classes_):
    label_indices = np.where(y_val == label_idx)[0]
    label_x_val = x_val[label_indices]
    label_y_val = y_val[label_indices]
    with torch.no_grad():
        label_x_val_tensor = torch.tensor(label_x_val, dtype=torch.float32).to(device)
        label_y_val_pred = model.predict(label_x_val_tensor)
        label_accuracy[label] = accuracy_score(label_y_val, label_y_val_pred)

In [12]:
for label, accuracy in label_accuracy.items():
    print(f"Label: {label}, Accuracy: {accuracy:.4f}")

Label: 0, Accuracy: 0.5659
Label: 1, Accuracy: 0.5566
Label: 2, Accuracy: 0.4494
Label: 3, Accuracy: 0.5866
Label: 4, Accuracy: 0.8036
Label: 5, Accuracy: 0.7563


In [13]:
with torch.no_grad():
    x_test_tensor = torch.tensor(x_test, dtype=torch.float32).to(device)
    y_test_pred = model.predict(x_test_tensor)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print(f"Testing Accuracy: {test_accuracy:.4f}")

Testing Accuracy: 0.6262
